# Tutorial on regexmodel

## Setup and installation

If you haven't installed `regexmodel` yet, including the optional dependencies, do so now:

In [1]:
# %pip install git+https://github.com/sodascience/regexmodel.git[tutorial]

Normally we would already have data that we want to model and synthesize, but for this tutorial we will use the faker package to generate that data for us. We will use fake email addresses.

In [2]:
from faker import Faker

fake = Faker("en")
Faker.seed(12345)
email_addresses = [fake.ascii_email() for _ in range(1000)]
email_addresses[:10]

['wilkinsleah@yahoo.com',
 'ashlee25@sawyer.net',
 'michaelrubio@schmidt.biz',
 'jamiemiller@morris-ellis.com',
 'smithelizabeth@gmail.com',
 'cory61@lane.net',
 'hallmorgan@gmail.com',
 'brianna82@petty.com',
 'michaelpeterson@morris-williams.info',
 'lvang@barton.com']

## Modeling the structured strings

Now we will use the regexmodel package to model the data:

In [3]:
from regexmodel import RegexModel

model = RegexModel.fit(email_addresses)

Let's first see how the good the model is by synthesizing new email addresses:

In [4]:
[model.draw() for _ in range(10)]

['smwf@hewpky.com',
 'quumzyrfto@prro.com',
 'syrhnuh54@atkz.net',
 'vhxnbazi@bbmjxhn-boar.org',
 'fjjphdhk39@qkte.com',
 'cqnnkfmehasy@bwdl.com',
 'jzdfifaogkatzozog@vnhkuu-fnaba.com',
 'ismcjjgzamxuife@fsasucahe.com',
 'jjswmuprnfx@lyhxck.com',
 'tzooc@vvranncgb.com']

While certainly not perfect, it certainly isn't so bad either, given that we have given the model only positive examples!

Now let's look at the serialization of the model:

In [5]:
model.serialize()

[{'regex': '[a-z]{3,18}[0-9]{2,2}[@][a-z]{4,9}[\\\\.][c][o][m]',
  'weights': [1000, 259, 259, 791, 746, 705, 705, 705],
  'side_branches_before': [{'i_branch': 0, 'data': {'weight': 466}},
   {'i_branch': 2,
    'data': {'regex': '[a-z]{4,18}',
     'weights': [533],
     'side_branches_before': [{'i_branch': 0, 'data': {'weight': 533}}],
     'side_branches_after': []}},
   {'i_branch': 2, 'data': {'weight': 1}}],
  'side_branches_after': [{'i_branch': 0,
    'data': {'regex': '[@][a-z]{3,9}[\\\\-][a-z]{4,9}[\\\\.][c][o][m]',
     'weights': [207, 207, 120, 119, 118, 70, 70, 70],
     'side_branches_before': [],
     'side_branches_after': [{'i_branch': 1,
       'data': {'regex': '[\\\\.][b][i][z]',
        'weights': [86, 26, 26, 26],
        'side_branches_before': [],
        'side_branches_after': [{'i_branch': 0,
          'data': {'regex': '[i][n][f][o]',
           'weights': [23, 23, 23, 23],
           'side_branches_before': [],
           'side_branches_after': [{'i_branc

The serialization might seem overwhelming at first, but the first regex (`[a-z]{3,18}[0-9]{2,2}[@][a-z]{4,9}[\\\\.][c][o][m]`) is usually the most important one. We call this the main branch. On this main branch, there will be side branches, for example for ".info" and ".biz" email addresses.

## Modeling performance

There are also some modeling statistics that can be computed. Note that computing these can take a while depending on your computer.

In [6]:
model.fit_statistics(email_addresses)

{'failed': 18,
 'success': 982,
 'n_tot_char': 22023,
 'n_char_success': 21582,
 'n_parameters': 121,
 'avg_log_like_per_char': -2.8483489023604314,
 'avg_log_like_pc_success': -2.7654002316121136}

What the `fit_statistics` method does is to retrace back whether an email address that is given to it (e.g. johndoe@example.com) has a non-zero probability to be generated by the regex model. As we can see above, there were 18 email addresses in the list that have a probability of 0 to be generated by the model, while the overwhelming majority (982) can be generated with the fitted model.

The value `n_parameters` gives the number of nodes in the model, and is thus an indicator of the complexity of the model. This is also correlated with the fit taking longer. We can influence this parameter during fitting by setting the `count_thres` parameter. If we set that threshold higher, we generally have a lower number of parameters and better performance.

The statistic `avg_log_like_per_char` (average log-likelihood per character) shows how probable a value is on average per character. To understand this better, let's take a more simple example, where the regex is simply `\d{2,2}`. For this regex, the log likelihood is simply log(1/10\*1/10) = -2\*log(10). Since all values have 2 characters, the average log-likelihood per character is -log(10) ~= 2.30. For failed values (values that cannot be generated by the model), we use a penalty score of -log(1000) per character.

Ideally we want to have the lowest `n_parameters` (simplest model) with the highest `success` and the highest log-likelihood.

## Visualization

To more clearly understand how the graph looks like, we can plot the regex model using the `regex_model_to_pyvis` function. To retrace the paths that can be taken, first find the start node and find the main branch.

Note: PyVis doesnt work interactively in VSCode/Code OSS.

In [7]:
from regexmodel.visualization import regex_model_to_pyvis

net = regex_model_to_pyvis(model)
net.show("regex.html", notebook=True)

regex.html
